# VDS 데이터 웹크롤링

## 공공데이터 포털 웹크롤링

In [ ]:
import requests
from requests_toolbelt import MultipartEncoder
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
def post_vds(date):
    multipart = MultipartEncoder(fields={
        'dataSupplyDate': f'{date}',
        'dataSupplyYear': 'null',
        'dataSupplyMonth': 'null',
        'dataSupplyYearQ': 'null',
        'dataSupplyQuater': 'null',
        'dataSupplyYearY': 'null',
        'collectType': 'VDS',
        'dataType': '16',
        'collectCycle': '04',
        'supplyCycle': '01',
        'outFileName': f'vds_{date}.gz',
    })

    headers = {
        'Content-Type': multipart.content_type,
    }

    res = requests.post('http://data.ex.co.kr/portal/fdwn/log', headers=headers, data=multipart)

    return res.content

In [ ]:
date = '20220701'

In [ ]:
print(f"Start crawling {date}")

print("POST ...")
data = post_vds(f'{date}')
print("Decompressing ...")
decompressed_data = gzip.decompress(data)
print("Saving ...")
with open(f'vds_{date}.csv', 'wb') as w:
    w.write(decompressed_data)

print("Complete !")

In [ ]:
df = pd.read_csv(f'vds_{date}.csv', encoding='euc-kr')
df

## 데이터 확인

### 1. 점유율

In [ ]:
df['점유율'].describe()

- 점유율 데이터가 존재하지 않으면 -1 값이 채워짐을 확인
- 비율 데이터로, 최소 0 최대 100의 데이터를 가지는 것을 확인
- 한양대 측에서 **aggregation할 때 산술평균으로 진행하면 된다고 했음**

### 2. 평균속도

In [ ]:
df['평균속도'].describe()

- 평균속도 또한 데이터가 존재하지 않으면 -1 값이 채워짐을 확인 (음수인 경우 데이터 제외하면 될 듯)
- 최고 속도가 180km/h로 비정상적인 데이터는 확인되지 않음
- 한양대 측에서 aggregation할 때 산술평균으로 진행하면 된다고 했음

### 3. 교통량

In [ ]:
df['교통량'].describe()

- 교통량도 데이터가 존재하지 않으면 음수 데이터가 들어감을 확인
- 한양대 측에서 aggregation할 때 단순 합하면 된다고 했음

## 유효하지 않은 데이터 삭제
- 교통량이 -1인 행은 의미없는 값으로 제외

In [ ]:
df = df[df['교통량'] != -1]

In [ ]:
df['평균속도'].describe()

In [ ]:
df['점유율'].describe()

## 30초 단위 데이터를 5분 단위 데이터로 취합
- 우리 DB에서 사용하는 데이터와 크롤링한 데이터 사이를 비교

### 1. 평균속도에 대한 30초 단위 데이터 aggregation

- [국가법령정보센터 차량검지기(VDS) 성능평가 기준 1. 개요 가. 대상장비](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwi-xKiq-5CBAxUxh1YBHZlNA8gQFnoECBAQAQ&url=https%3A%2F%2Fwww.law.go.kr%2FflDownload.do%3FflSeq%3D72668643%26flNm%3D%255B%25EB%25B3%2584%25ED%2591%259C%2B4%255D%2B%25EC%25B0%25A8%25EB%259F%2589%25EA%25B2%2580%25EC%25A7%2580%25EA%25B8%25B0%2528VDS%2529%2B%25EC%2584%25B1%25EB%258A%25A5%25ED%258F%2589%25EA%25B0%2580%2B%25EA%25B8%25B0%25EC%25A4%2580&usg=AOvVaw0JvMC9hj7b-wJFfJSvLYjh&opi=89978449)
  - 교통량: 분석단위시간당 측정된 차량의 합
  - 속도: 분석단위시간당 측정된 차량의 속도를 산술평균한 값
  - 여기서, 속도와 교통량을 곱하면 산술평균하기 전에 30초 동안 측정된 속도의 총합이 나오며, 이들을 5분 동안의 데이터들에서 추출하여 다시 누적합
  - 이렇게 한 누적합된 속도를 5분동안 누적한 교통량으로 나누어 5분 동안에 측정된 차량의 속도를 산술평균한 값을 구할 수 있음

In [ ]:
def aggregate_speed(data: pd.DataFrame):
    if len(data) == 0:
        return -1

    speed_sum = np.inner(data['교통량'], data['평균속도']) # 각 vds 데이터에 대한 속도의 누적합을 구한다.
    count = sum(data['교통량']) # 교통량의 합을 구해 검지기를 지난 차량의 수를 구한다.

    if count == 0:
        return 0

    return speed_sum / count

### 2. 점유율에 대한 30초 단위 데이터 aggregation
- [국가법령정보센터 차량검지기(VDS) 성능평가 기준 1. 개요 가. 대상장비](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwi-xKiq-5CBAxUxh1YBHZlNA8gQFnoECBAQAQ&url=https%3A%2F%2Fwww.law.go.kr%2FflDownload.do%3FflSeq%3D72668643%26flNm%3D%255B%25EB%25B3%2584%25ED%2591%259C%2B4%255D%2B%25EC%25B0%25A8%25EB%259F%2589%25EA%25B2%2580%25EC%25A7%2580%25EA%25B8%25B0%2528VDS%2529%2B%25EC%2584%25B1%25EB%258A%25A5%25ED%258F%2589%25EA%25B0%2580%2B%25EA%25B8%25B0%25EC%25A4%2580&usg=AOvVaw0JvMC9hj7b-wJFfJSvLYjh&opi=89978449)
  - “점유시간”(초)이라 함은 VDS 기본성능평가에서 차량이 검지영역을 통과하는데 소요되는 시간을 말한다.
  - “점유율”(%)이라 함은 VDS 기본성능평가에서 분석 단위시간당 측정된 점유시간의 합을 백분율로 나타낸 것을 말한다.
- 결국, 점유율은 VDS를 운영하는 단위 시간(여기서는 30초) 동안 차량들이 그 위(VDS)를 얼마 시간동안 올라가 있었는지를 말한다.
- 이를 통해, 30초 동안의 점유 시간을 계산할 수 있고, 이를 모두 합하여 5분 동안의 점유율을 구할 수 있다.

In [ ]:
def aggregate_share(data: pd.DataFrame):
    if len(data) == 0:
        return -1

    time_sum = sum(data['점유율'] / 100 * 30) # 백분율을 비율로 환산, 30초 곱하여 단위시간당 측정된 점유시간의 합 계산
    time_total = 300 # 5분 == 300초

    return time_sum / time_total * 100

### 3. 교통량에 대한 30초 단위 데이터 aggregation

In [ ]:
def aggregate_traffic(data: pd.DataFrame):
    if len(data) == 0:
        return -1

    return sum(data['교통량']) # 단순 누적합

## 우리 DB에서 사용하는 5분 데이터와 값 비교

In [ ]:
sample_df = pd.read_csv('vds_sample.csv', encoding='euc-kr', header=None, names=[
    '도로번호',
    '도로명',
    '콘존ID',
    '구간명',
    '구간길이(m)',
    '기점종점방향구분코드',
    '집계일자',
    '집계시분',
    'VDS_ID',
    '차로유형구분코드',
    '교통량',
    '점유율',
    '평균속도',
    'Unknown',
], dtype={
    '집계시분': str
})
sample_df

In [ ]:
from datetime import datetime, timedelta

result = []

for idx, row in tqdm(sample_df.iterrows()):
    item = []

    vds_id = row['VDS_ID']
    item.append(vds_id) # VDS_ID

    _start = datetime.strptime(row['집계시분'] + '00', '%H%M%S')
    _end = _start + timedelta(minutes=5)

    start = int(_start.strftime('%H%M%S'))
    end = int(_end.strftime('%H%M%S'))

    temp_df = df.query(f"VDS_ID == '{vds_id}' and 수집시분초 >= {start} and 수집시분초 < {end}")
    
    item.append(aggregate_traffic(temp_df)) # 취합교통량
    item.append(row['교통량']) # 5분교통량

    item.append(aggregate_share(temp_df)) # 취합점유율
    item.append(row['점유율']) # 5분점유율

    item.append(aggregate_speed(temp_df)) # 취합평균속도
    item.append(row['평균속도']) # 5분평균속도

    result.append(item)

result_df = pd.DataFrame(result, columns=['VDS_ID', '취합교통량', '5분교통량', '취합점유율', '5분점유율', '취합평균속도', '5분평균속도'])
result_df

In [ ]:
result_df.to_csv('result.csv', index=False)